In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# 한글 폰트 깨짐 방지
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 19 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (8,981 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletyp

In [6]:
import pandas as pd
import numpy as np
import json

In [ ]:
# 파일 불러오기
train = pd.read_json('data/train.json') 
val = pd.read_json('data/val.json') 
test = pd.read_json('data/test.json')
song_meta = pd.read_json('data/song_meta.json')

## 1. 곡 별 메타 데이터: song_meta.json

**장르 구분 및 데이터 확인**

In [ ]:
song_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707989 entries, 0 to 707988
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   song_gn_dtl_gnr_basket  707989 non-null  object
 1   issue_date              707989 non-null  int64 
 2   album_name              707985 non-null  object
 3   album_id                707989 non-null  int64 
 4   artist_id_basket        707989 non-null  object
 5   song_name               707989 non-null  object
 6   song_gn_gnr_basket      707989 non-null  object
 7   artist_name_basket      707989 non-null  object
 8   id                      707989 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 48.6+ MB


In [ ]:
song_meta.head()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4


In [ ]:
# 변수명 변경
song_meta.rename(columns={'song_gn_dtl_gnr_basket' : '곡 세부 장르 리스트',\
                         'issue_date' : '발매일',\
                         'album_name' : '앨범 명',\
                         'album_id' : '앨범 ID',\
                         'artist_id_basket' : '아티스트 ID 리스트',\
                         'song_name' : '곡 제목',\
                         'song_gn_gnr_basket' : '곡 장르 리스트',\
                         'artist_name_basket' : '아티스트 리스트',\
                         'id' : '곡 ID'}, inplace=True)
song_meta.head()

,곡 세부 장르 리스트,발매일,앨범 명,앨범 ID,아티스트 ID 리스트,곡 제목,곡 장르 리스트,아티스트 리스트,곡 ID
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4


In [ ]:
# song_meta에 있는 장르 대분류
genre_list = []
song_meta['곡 장르 리스트'].apply(lambda x : genre_list.extend(x))
unique_genre_list = set(genre_list)
unique_genre_list

{'GN0100',
 'GN0200',
 'GN0300',
 'GN0400',
 'GN0500',
 'GN0600',
 'GN0700',
 'GN0800',
 'GN0900',
 'GN1000',
 'GN1100',
 'GN1200',
 'GN1300',
 'GN1400',
 'GN1500',
 'GN1600',
 'GN1700',
 'GN1800',
 'GN1900',
 'GN2000',
 'GN2100',
 'GN2200',
 'GN2300',
 'GN2400',
 'GN2500',
 'GN2600',
 'GN2700',
 'GN2800',
 'GN2900',
 'GN9000'}

In [ ]:
len(unique_genre_list) # 대분류는 30개

30

In [ ]:
# song_meta에 있는 소분류
genre_detail_list = []
song_meta['곡 세부 장르 리스트'].apply(lambda x : genre_detail_list.extend(x))
unique_genre_detail_list = set(genre_detail_list)
unique_genre_detail_list

{'GN0101',
 'GN0102',
 'GN0103',
 'GN0104',
 'GN0105',
 'GN0201',
 'GN0202',
 'GN0203',
 'GN0204',
 'GN0205',
 'GN0301',
 'GN0302',
 'GN0303',
 'GN0304',
 'GN0401',
 'GN0402',
 'GN0403',
 'GN0501',
 'GN0502',
 'GN0503',
 'GN0504',
 'GN0505',
 'GN0506',
 'GN0507',
 'GN0508',
 'GN0509',
 'GN0601',
 'GN0602',
 'GN0603',
 'GN0604',
 'GN0605',
 'GN0606',
 'GN0701',
 'GN0702',
 'GN0703',
 'GN0704',
 'GN0705',
 'GN0706',
 'GN0707',
 'GN0708',
 'GN0709',
 'GN0710',
 'GN0801',
 'GN0802',
 'GN0803',
 'GN0804',
 'GN0805',
 'GN0901',
 'GN0902',
 'GN0903',
 'GN0904',
 'GN0905',
 'GN0906',
 'GN0907',
 'GN0908',
 'GN1001',
 'GN1002',
 'GN1003',
 'GN1004',
 'GN1005',
 'GN1006',
 'GN1007',
 'GN1008',
 'GN1009',
 'GN1010',
 'GN1011',
 'GN1012',
 'GN1013',
 'GN1014',
 'GN1101',
 'GN1102',
 'GN1103',
 'GN1104',
 'GN1105',
 'GN1106',
 'GN1107',
 'GN1108',
 'GN1109',
 'GN1201',
 'GN1202',
 'GN1203',
 'GN1204',
 'GN1205',
 'GN1206',
 'GN1207',
 'GN1208',
 'GN1209',
 'GN1301',
 'GN1302',
 'GN1303',
 'GN1304',

In [ ]:
len(unique_genre_detail_list) # 소분류는 219개

219

In [ ]:
song_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707989 entries, 0 to 707988
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   곡 세부 장르 리스트  707989 non-null  object
 1   발매일          707989 non-null  int64 
 2   앨범 명         707985 non-null  object
 3   앨범 ID        707989 non-null  int64 
 4   아티스트 ID 리스트  707989 non-null  object
 5   곡 제목         707989 non-null  object
 6   곡 장르 리스트     707989 non-null  object
 7   아티스트 리스트     707989 non-null  object
 8   곡 ID         707989 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 48.6+ MB


In [ ]:
null = song_meta[song_meta['곡 세부 장르 리스트'].apply(len) == 0]
null # 4037개

,곡 세부 장르 리스트,발매일,앨범 명,앨범 ID,아티스트 ID 리스트,곡 제목,곡 장르 리스트,아티스트 리스트,곡 ID
21,[],20200331,WHY,10410508,[417985],WHY,[],[4minute],21
262,[],20150318,노년의 건강을 지켜주는 음악 처방전 (부제 : 휴식 때 들으면 좋은 음악친구),2309524,[726909],Bio Sound `Breathing & Big Bell`,[GN9000],[차병원],262
399,[],20200106,뽀로로 겨울 동요,10372098,[1703695],초코 초코 핫초코,[],[아이코닉스 (ICONIX)],399
454,[],20191108,MAGDALENE,10349014,[788767],thousand eyes,[GN1100],[FKA Twigs],454
803,[],20040901,Tropical Lullaby,153435,[848353],Kalua Lullaby,[],[The Moonlighters],803
...,...,...,...,...,...,...,...,...,...
706502,[],20110831,엄마와 아가와의 행복한 대화 태교음악 뉴에이지,2009854,[561734],Love Love Love,[GN9000],[해피타임뮤직],706502
706716,[],20200109,Have It All,10373946,[2762722],Have It All,[GN9000],[Renegade],706716
707201,[],20200115,모두의 MR반주 123,10376105,[770787],Into the Unknown (겨울왕국 2 OST) (Melody MR),[GN9000],[모두의MR],707201
707759,[],20200225,낭만닥터 김사부 2 OST,10393897,"[27619, 726783]",Emergency,[],"[전창엽, 마마고릴라]",707759


In [ ]:
null = song_meta[song_meta['곡 장르 리스트'].apply(len) == 0]
null # 1059개

,곡 세부 장르 리스트,발매일,앨범 명,앨범 ID,아티스트 ID 리스트,곡 제목,곡 장르 리스트,아티스트 리스트,곡 ID
21,[],20200331,WHY,10410508,[417985],WHY,[],[4minute],21
399,[],20200106,뽀로로 겨울 동요,10372098,[1703695],초코 초코 핫초코,[],[아이코닉스 (ICONIX)],399
803,[],20040901,Tropical Lullaby,153435,[848353],Kalua Lullaby,[],[The Moonlighters],803
1089,[],20190924,66,10331496,[2747498],How are you,[],[유노이아 (Eunoia)],1089
2588,[],20080709,명상태교음악 `평온` Vol.4,392967,[2727],Dvorak : Humoresque No.7 Op.101,[],[Various Artists],2588
...,...,...,...,...,...,...,...,...,...
701617,[],20191205,Special Christmas (크리스마스 캐롤),10361888,[28566],The Christmas Spell,[],[Peggy Lee],701617
703137,[],20200417,BEIGE 0.5,10419112,[873162],손톱,[],[Kid Milli],703137
704799,[],20200331,Love Tension,10410516,[417985],Love Tension,[],[4minute],704799
705983,[],20200106,TREASURE EPILOGUE : Action To Answer,10371984,[2398260],지평선 (Horizon),[],[ATEEZ (에이티즈)],705983


### 2. 곡 장르 데이터 확인: genre_gn_all.json

In [ ]:
# 시리즈 타입으로 데이터 
genre = pd.read_json('data/genre_gn_all.json', typ='series')

In [ ]:
genre

GN0100       발라드
GN0101    세부장르전체
GN0102       '80
GN0103       '90
GN0104       '00
           ...  
GN2900       뮤지컬
GN2901    세부장르전체
GN2902     국내뮤지컬
GN2903     국외뮤지컬
GN3000     크리스마스
Length: 254, dtype: object

In [ ]:
#  장르 코드: gnr_code, 장르명: gnr_name
# 시리즈를 데이터 프레임화 / 인덱스 재설정 /  컬럼명 변경
genre = pd.DataFrame(genre).reset_index().rename(columns = {'index' : 'gnr_code', 0:'gnr_name'})
genre

,gnr_code,gnr_name
0,GN0100,발라드
1,GN0101,세부장르전체
2,GN0102,'80
3,GN0103,'90
4,GN0104,'00
...,...,...
249,GN2900,뮤지컬
250,GN2901,세부장르전체
251,GN2902,국내뮤지컬
252,GN2903,국외뮤지컬


In [ ]:
genre.columns

Index(['gnr_code', 'gnr_name'], dtype='object')

In [ ]:
genre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   gnr_code  254 non-null    object
 1   gnr_name  254 non-null    object
dtypes: object(2)
memory usage: 4.1+ KB


* 장르 코드 및 장르명은 총 254개 / Null값 없음


#### 대분류 및 소분류 장르 코드 확인
* 대분류: gnr_code에서 뒷자리가 00인 코드
* 소분류: gnr_code에서 뒷자리가 00이 아닌 코드

In [ ]:
# 대분류 장르 코드 추출
genre_big_cat = genre[genre['gnr_code'].apply(lambda x: str(x)[-2:]) == '00']
genre_big_cat

,gnr_code,gnr_name
0,GN0100,발라드
6,GN0200,댄스
12,GN0300,랩/힙합
18,GN0400,R&B/Soul
22,GN0500,인디음악
32,GN0600,록/메탈
39,GN0700,성인가요
50,GN0800,포크/블루스
56,GN0900,POP
65,GN1000,록/메탈


In [ ]:
len(genre_big_cat) # 장르 대 분류는 30개

30

In [ ]:
# 소분류 장르 코드 추출
genre_small_cat = genre[genre['gnr_code'].apply(lambda x: str(x)[-2:]) != '00']
genre_small_cat

,gnr_code,gnr_name
1,GN0101,세부장르전체
2,GN0102,'80
3,GN0103,'90
4,GN0104,'00
5,GN0105,'10-
...,...,...
247,GN2805,숙면유도
248,GN2806,반려동물
250,GN2901,세부장르전체
251,GN2902,국내뮤지컬


장르 코드 트리(Tree)
* 대분류 - 소분류 활용한 장르 코드 트리 구성

In [ ]:
# 1. 장르 대분류와 소분류를 조인하기 위한 공통 컬럼 join_code 생성
# 기존 데이터프레임 객체에 열 추가해서 새로운 개체 반환하는 형식
genre_big_cat =  genre_big_cat.assign(join_code = genre_big_cat['gnr_code'].str[0:4])
genre_small_cat = genre_small_cat.assign(join_code = genre_small_cat['gnr_code'].str[0:4])

In [ ]:
# 2. Merge 
# join_code를 공통 컬럼으로 해서 left-join으로 big_gerne와 small_genre 테이블을 Merge 
genre_tree = pd.merge(genre_big_cat, genre_small_cat, how = 'left', on = 'join_code')

In [ ]:
# join_code 열 삭제
genre_tree.drop(['join_code'], axis=1, inplace=True)

In [ ]:
# 컬럼명을 보기 쉽게 수정
genre_tree.columns = ['대분류 장르코드', '대분류 장르명', '소분류 장르코드', '소분류 장르명']

In [ ]:
genre_tree

,대분류 장르코드,대분류 장르명,소분류 장르코드,소분류 장르명
0,GN0100,발라드,GN0101,세부장르전체
1,GN0100,발라드,GN0102,'80
2,GN0100,발라드,GN0103,'90
3,GN0100,발라드,GN0104,'00
4,GN0100,발라드,GN0105,'10-
...,...,...,...,...
220,GN2800,뮤직테라피,GN2806,반려동물
221,GN2900,뮤지컬,GN2901,세부장르전체
222,GN2900,뮤지컬,GN2902,국내뮤지컬
223,GN2900,뮤지컬,GN2903,국외뮤지컬


In [ ]:
genre_tree.info()
# 대분류는 225개 / 소분류는 224개(null값이 하나 있음)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225 entries, 0 to 224
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   대분류 장르코드  225 non-null    object
 1   대분류 장르명   225 non-null    object
 2   소분류 장르코드  224 non-null    object
 3   소분류 장르명   224 non-null    object
dtypes: object(4)
memory usage: 8.8+ KB


In [ ]:
genre_tree[genre_tree['소분류 장르코드'].isnull()]

,대분류 장르코드,대분류 장르명,소분류 장르코드,소분류 장르명
224,GN3000,크리스마스,NaN,NaN


In [ ]:
# 대분류에 포함된 소분류 장르 확인하기
genre_tree[genre_tree['대분류 장르코드'] == 'GN0100']

,대분류 장르코드,대분류 장르명,소분류 장르코드,소분류 장르명
0,GN0100,발라드,GN0101,세부장르전체
1,GN0100,발라드,GN0102,'80
2,GN0100,발라드,GN0103,'90
3,GN0100,발라드,GN0104,'00
4,GN0100,발라드,GN0105,'10-


### 3. 학습데이터 tain.json

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115071 entries, 0 to 115070
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   tags          115071 non-null  object
 1   id            115071 non-null  int64 
 2   plylst_title  115071 non-null  object
 3   songs         115071 non-null  object
 4   like_cnt      115071 non-null  int64 
 5   updt_date     115071 non-null  object
dtypes: int64(2), object(4)
memory usage: 5.3+ MB


In [ ]:
train.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000


In [ ]:
# train 데이터에 태그 개수 확인
tag_list = []
train['tags'].apply(lambda x : tag_list.extend(x))
unique_tag_list = set(tag_list)
unique_tag_list

{'',
 '올드팝리믹스',
 'MTV',
 '겨울비와함께',
 '게리멀리건',
 'Ciara',
 '아리아나_그란데',
 'ShapeOfYou',
 '인내는',
 '서커스',
 '킹도',
 '번안가요',
 '병장노래',
 '오래된연인',
 '헤이트풀8',
 '쇼머니',
 '일곱시쯤',
 'gorgeous',
 '흥하자',
 '응팔',
 '낭만클래식',
 '가요',
 '게릴라',
 '내가듣고싶어서만든플레이리스트',
 '흐린날',
 '꽃보다청춘아프리카',
 '제3의매력',
 '대만팝',
 'Kashif',
 '딕시칙스',
 '주말엔',
 '봄을부르는노래',
 '2월',
 '신선함을',
 '고속도로',
 '독일전승리',
 '눈내리는날',
 '매드홀릭',
 '달라',
 'MayuWakisaka',
 '에어컨이짱이야',
 '앱신트',
 'IrvingBerlin',
 '느낌있는곡',
 '조승연인라',
 '산들',
 '빵음악',
 '잭헤론',
 '센티멘탈',
 '업로드용',
 '한걸음',
 '해외힙합',
 '하반기',
 'TOP20',
 '나의태그',
 'GirlCrush',
 '예쁜밤',
 '마법',
 '바수르',
 '무채색',
 '음악추천',
 '비가내리면',
 'SS501',
 '가슴아픈',
 '잠깨우기',
 '코끝시린',
 '내',
 '여러분위기',
 '격정',
 '일찍일어나는새',
 '돈꽃',
 '잔잔하게',
 '모여라',
 '최고야',
 '율동찬양',
 '가요재즈',
 '영앤리치',
 '어머이건들어야해',
 '어린이찬양',
 'MarkKnopfler',
 '멜랑꼴리한',
 '공부하기싫은사람',
 '여자보컬CCM',
 '숨겨진_명곡들',
 '센스넘친다는',
 '마일즈데이비스',
 '마음을_차분하게해줄_음악',
 '많이들어본노래',
 '가을추천곡',
 '루에드랑고르',
 '센치한척',
 '달달한노래',
 '배추',
 '브로콜리너마저',
 '글라스톤베리',
 '대부분',
 '단체곡들',
 '그린데이내한',
 '그래비티',
 '스틸우지',
 '팝CCM',
 '큐팁',

In [ ]:
len(unique_tag_list)

29160

In [ ]:
# 업데이트 연 월에 대한 컬럼 생성
train['year'] = train['updt_date'].apply(lambda x : x.split('-')[0])
train['month'] = train['updt_date'].apply(lambda x : x.split('-')[1])

In [ ]:
train.head()

,tags,id,plylst_title,songs,like_cnt,updt_date,year,month
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,2013,12
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,2014,12
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,2017,08
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,2019,12
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,2011,10


In [ ]:
train.rename(columns={'tags' : '태그 리스트',
                      'id' : '플레이리스트 ID', 
                      'plylst_title' : '플레이리스트 제목', 
                      'songs' : '곡 리스트', 
                      'like_cnt' : '좋아요 개수', 
                      'updt_date' : '수정 날짜', 
                      'year' : '수정 연도',
                      'month' : '수정 월'}, inplace=True)
train.head()

,태그 리스트,플레이리스트 ID,플레이리스트 제목,곡 리스트,좋아요 개수,수정 날짜,수정 연도,수정 월
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,2013,12
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,2014,12
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,2017,08
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,2019,12
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,2011,10


In [ ]:
# 플레이리스트 ID, 수록곡 추출
plylst_song = train[['플레이리스트 ID', '곡 리스트']]
plylst_song

,플레이리스트 ID,곡 리스트
0,61281,"[525514, 129701, 383374, 562083, 297861, 13954..."
1,10532,"[432406, 675945, 497066, 120377, 389529, 24427..."
2,76951,"[83116, 276692, 166267, 186301, 354465, 256598..."
3,147456,"[394031, 195524, 540149, 287984, 440773, 10033..."
4,27616,"[159327, 553610, 5130, 645103, 294435, 100657,..."
...,...,...
115066,120325,"[429629, 441511, 612106, 516359, 691768, 38714..."
115067,106976,"[321330, 216057, 534472, 240306, 331098, 23288..."
115068,11343,"[50512, 249024, 250608, 371171, 229942, 694943..."
115069,131982,"[533534, 608114, 343608, 417140, 609009, 30217..."


In [ ]:
# 플레이리스트와 곡 리스트를 세로로 이어 붙여 새로운 객체 생성
# 플레이리스트 ID를 수록곡 수만큼 반복해서 플레이리스트 ID 컬럼에 붙인 A
# 곡 리스트를 하나씩 갈라서 한 리스트에 담은 B를 세로로 붙이기 

playlist_song = np.dstack(
    (
        np.repeat(plylst_song['플레이리스트 ID'].values, list(map(len, plylst_song['곡 리스트']))),
        # 곡 리스트 개수(len)만큼 플레이리스트 ID를 반복하여 만든 어레이 
        np.concatenate(plylst_song['곡 리스트'].values)
        # 곡 리스트 값
    )
)
playlist_song

array([[[ 61281, 525514],
        [ 61281, 129701],
        [ 61281, 383374],
        ...,
        [100389, 640239],
        [100389,  13759],
        [100389, 154078]]])

In [ ]:
# 데이터 프레임 생성
playlist_song_df = pd.DataFrame(data = playlist_song[0])
playlist_song_df.rename(columns = {0:'플레이리스트 ID',
                                   1 : '곡 리스트'}, inplace=True)
playlist_song_df

,플레이리스트 ID,곡 리스트
0,61281,525514
1,61281,129701
2,61281,383374
3,61281,562083
4,61281,297861
...,...,...
5285866,100389,111365
5285867,100389,51373
5285868,100389,640239
5285869,100389,13759


In [ ]:
# 플레이리스트 아이디(id)와 매핑된 태그(tags) 추출
plylst_tag = train[['플레이리스트 ID', '태그 리스트']]
plylst_tag

,플레이리스트 ID,태그 리스트
0,61281,[락]
1,10532,"[추억, 회상]"
2,76951,"[까페, 잔잔한]"
3,147456,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,..."
4,27616,[댄스]
...,...,...
115066,120325,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]"
115067,106976,[일렉]
115068,11343,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]"
115069,131982,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]"


In [ ]:
# 플레이리스트와 태그 리스트를 세로로 이어 붙여 새로운 객체 생성
# 플레이리스트 ID를 태그 수만큼 반복해서 플레이리스트 ID 컬럼에 붙인 A
# 태그 리스트를 하나씩 갈라서 한 리스트에 담은 B를 세로로 붙이기 

playlist_tag = np.dstack(
    (
        np.repeat(plylst_tag['플레이리스트 ID'].values, list(map(len, plylst_tag['태그 리스트']))),
        # 태그 리스트 개수(len)만큼 플레이리스트 ID를 반복하여 만든 어레이 
        np.concatenate(plylst_tag['태그 리스트'].values)
        # 태그 리스트 값
    )
)
playlist_tag

array([[['61281', '락'],
        ['10532', '추억'],
        ['10532', '회상'],
        ...,
        ['100389', '팝송추천'],
        ['100389', '팝송'],
        ['100389', '팝송모음']]], dtype='<U21')

In [ ]:
# 데이터 프레임 생성
playlist_tag_df = pd.DataFrame(data = playlist_tag[0])
playlist_tag_df.rename(columns = {0:'플레이리스트 ID',
                                   1 : '태그'}, inplace=True)

In [ ]:
# 플레이리스트 ID를 태그 테이블로 추가
playlist_tag_df['태그 테이블'] = playlist_tag_df['플레이리스트 ID']
playlist_tag_df

,플레이리스트 ID,태그,태그 테이블
0,61281,락,61281
1,10532,추억,10532
2,10532,회상,10532
3,76951,까페,76951
4,76951,잔잔한,76951
...,...,...,...
476326,131982,퇴근길,131982
476327,100389,노래추천,100389
476328,100389,팝송추천,100389
476329,100389,팝송,100389


In [ ]:
# 유니크한 곡 수
train_unique_song_cnt = playlist_song_df['곡 리스트'].nunique()
# 유니크한 태그 수
traint_unique_tag_cnt = playlist_tag_df['태그'].nunique()

print('train 데이터의 곡 수: {}'.format(train_unique_song_cnt))
print('train 데이터의 태그 수: {}'.format(traint_unique_tag_cnt))

train 데이터의 곡 수: 615142
train 데이터의 태그 수: 29160


## 1주차 - 데이터 전처리 (경희씨)

In [ ]:
song_meta

,곡 세부 장르 리스트,발매일,앨범 명,앨범 ID,아티스트 ID 리스트,곡 제목,곡 장르 리스트,아티스트 리스트,곡 ID
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4
...,...,...,...,...,...,...,...,...,...
707984,[GN2001],19991219,The Best Best Of The Black President,65254,[166499],Coffin For Head Of State,[GN2000],[Fela Kuti],707984
707985,[GN0901],19860000,True Colors,44141,[11837],Change Of Heart,[GN0900],[Cyndi Lauper],707985
707986,"[GN0105, GN0101]",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,[437],스치듯 안녕,[GN0100],[윤종신],707986
707987,"[GN1807, GN1801]",20131217,명상의 시간을 위한 뉴에이지 음악,2221722,[729868],숲의 빛,[GN1800],[Nature Piano],707987


In [ ]:
# song_meta의 song_gn_gnr_basket column에는 0개부터 6개까지의 장르가 들어있음
song_meta['곡 장르 리스트'].apply(len).unique()

array([1, 2, 0, 3, 4, 5, 6])

In [ ]:
# 2020년도 데이터까지만 있어야하는데 2022년 데이터가 들어있음
# 확인 결과 둘 다 20220113이 아니라 20200113임 수정해주기
song_meta[song_meta['발매일'] > 20200423]

,곡 세부 장르 리스트,발매일,앨범 명,앨범 ID,아티스트 ID 리스트,곡 제목,곡 장르 리스트,아티스트 리스트,곡 ID
141185,[GN1801],20220113,고요한 밤바다 구경하기 [여수 바다],10403230,[2737142],잔잔한 바람이 불어와 (There is a gentle breeze),[GN1800],[무드홀릭 (Moodholic)],141185
448286,[GN1801],20220113,고요한 밤바다 구경하기 [여수 바다],10403230,[2737142],여수 바다 (Yeosu sea),[GN1800],[무드홀릭 (Moodholic)],448286


In [ ]:
song_meta.loc[:,'발매일'][song_meta['발매일'] > 20200423] = 20200113

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
# 값을 바꿔줘서 2020년 이후 데이터 없음
song_meta[song_meta['발매일'] > 20200423]

,곡 세부 장르 리스트,발매일,앨범 명,앨범 ID,아티스트 ID 리스트,곡 제목,곡 장르 리스트,아티스트 리스트,곡 ID


In [ ]:
# 앨범 이름이 비어있는 곡 4곡 존재
song_meta[song_meta['앨범 명'].isnull()]
# 사랑바보
# 슈퍼스타K2 Up To 11
# null
# 슈퍼스타K2 Up To 11

,곡 세부 장르 리스트,발매일,앨범 명,앨범 ID,아티스트 ID 리스트,곡 제목,곡 장르 리스트,아티스트 리스트,곡 ID
143209,"[GN0501, GN0601, GN0503, GN0606, GN0509]",20150522,None,2318271,[726168],사랑바보,"[GN0500, GN0600]",[민은서밴드],143209
291631,"[GN0105, GN0101]",20101108,None,1065178,[28772],빗속에서,[GN0100],[존박],291631
431110,"[GN0509, GN0601, GN0503, GN0606, GN0501]",20200208,None,10385717,[2203064],화면,"[GN0500, GN0600]",[Oowl Hannal (우울 한날)],431110
552088,"[GN0105, GN0601]",20101108,None,1065178,[28263],본능적으로 (Feat. Swings),"[GN0600, GN0100]",[강승윤],552088


In [ ]:
song_meta.loc[:,'앨범 명'][song_meta['곡 ID'] == 143209] = '사랑바보'
song_meta.loc[:,'앨범 명'][(song_meta['곡 ID'] == 291631) | (song_meta['곡 ID'] == 552088)] = '슈퍼스타K2 Up To 11'
song_meta.loc[:,'앨범 명'][song_meta['곡 ID'] == 431110] = 'null'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid 

In [ ]:
song_meta[song_meta['앨범 명'].isnull()]

,곡 세부 장르 리스트,발매일,앨범 명,앨범 ID,아티스트 ID 리스트,곡 제목,곡 장르 리스트,아티스트 리스트,곡 ID


In [ ]:
train

,태그 리스트,플레이리스트 ID,플레이리스트 제목,곡 리스트,좋아요 개수,수정 날짜,수정 연도,수정 월
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,2013,12
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,2014,12
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,2017,08
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,2019,12
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,2011,10
...,...,...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000,2020,04
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000,2015,12
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000,2019,08
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000,2019,10


In [ ]:
# 플레이리스트 제목 중복 merge
# train # 115071
train['플레이리스트 제목'].duplicated().sum()

4540

In [ ]:
# train[train['plylst_title'].duplicated()]
train[train['플레이리스트 제목'] == '미리 메리 크리스마스']

,태그 리스트,플레이리스트 ID,플레이리스트 제목,곡 리스트,좋아요 개수,수정 날짜,수정 연도,수정 월
241,[겨울],43159,미리 메리 크리스마스,"[326204, 174967, 117169, 703323, 638488, 59252...",17,2015-11-22 01:28:30.000,2015,11
1408,"[겨울, 크리스마스, 캐럴]",66941,미리 메리 크리스마스,"[291933, 162896, 61771, 62574, 279545, 667770,...",1,2020-02-20 11:03:33.000,2020,02
60917,[겨울],134554,미리 메리 크리스마스,"[326204, 105433, 388560, 469093, 186507, 26230...",11,2015-09-15 20:21:02.000,2015,09
82932,"[겨울_공통, 연말, 따듯한, 로멘틱, 크리스마스]",35336,미리 메리 크리스마스,"[290987, 257231, 243721, 326204, 166884, 13796...",4,2020-04-20 03:00:40.000,2020,04
90545,"[클래식, 캐롤]",15960,미리 메리 크리스마스,"[656246, 101804, 413694, 684941, 385279, 43796...",104,2016-12-06 17:17:18.000,2016,12
100082,[팝],17688,미리 메리 크리스마스,"[387859, 419281, 47797, 393595, 529003, 407505...",5,2014-11-19 15:53:35.000,2014,11


In [ ]:
train[train['플레이리스트 제목'] == '부활']

,태그 리스트,플레이리스트 ID,플레이리스트 제목,곡 리스트,좋아요 개수,수정 날짜,수정 연도,수정 월
709,[락],101001,부활,"[652900, 597435, 549966, 24432, 388523, 225905...",7,2011-03-06 02:24:00.000,2011,03
1187,[락],135125,부활,"[82568, 354259, 647864, 675040, 134381, 281364...",9,2011-09-19 11:00:17.000,2011,09
90938,[락],64763,부활,"[82568, 647864, 675040, 701638, 230194, 134381...",0,2011-09-19 10:47:30.000,2011,09


In [ ]:
def f(x):
  d = {}
  d['태그 리스트'] = list(set(sum(x['태그 리스트'], [])))
  d['플레이리스트 ID'] = x['플레이리스트 ID'].max()
  d['플레이리스트 제목'] = x['플레이리스트 제목'].max()
  d['곡 리스트'] = list(set(sum(x['곡 리스트'], [])))
  d['좋아요 개수'] = x['좋아요 개수'].sum()
  d['수정 날짜'] = x['수정 날짜'].max()
  return pd.Series(d, index=['태그 리스트', '플레이리스트 ID', '플레이리스트 제목', '곡 리스트', '좋아요 개수', '수정 날짜'])
unique_train = train.groupby(by='플레이리스트 제목').apply(f)
unique_train[unique_train['플레이리스트 제목'] == '부활']

,태그 리스트,플레이리스트 ID,플레이리스트 제목,곡 리스트,좋아요 개수,수정 날짜
플레이리스트 제목,,,,,,
부활,[락],135125,부활,"[371463, 82568, 529, 579858, 281364, 626581, 4...",16,2011-09-19 11:00:17.000


In [ ]:
unique_train[unique_train['플레이리스트 제목'] == '미리 메리 크리스마스']

,태그 리스트,플레이리스트 ID,플레이리스트 제목,곡 리스트,좋아요 개수,수정 날짜
플레이리스트 제목,,,,,,
미리 메리 크리스마스,"[클래식, 캐럴, 팝, 겨울_공통, 크리스마스, 로멘틱, 연말, 따듯한, 캐롤, 겨울]",134554,미리 메리 크리스마스,"[210432, 624648, 243721, 468494, 399888, 41422...",142,2020-04-20 03:00:40.000


In [ ]:
# 플레이리스트 수록곡 한개 이하인 플레이리스트 drop
unique_train.drop(unique_train[unique_train['곡 리스트'].apply(len) <= 1].index, inplace=True)

In [ ]:
unique_train.to_csv('train_1.csv',encoding='utf-8')

## 1주차 - 데이터 전처리 (혜영)
플레이리스트 명에서 단어 추출 -> 태그 추가

In [ ]:
# 파일 불러오기
unique_train = pd.read_csv('train_1.csv') 

In [ ]:
unique_train.head()

,플레이리스트 제목,태그 리스트,플레이리스트 ID,플레이리스트 제목.1,곡 리스트,좋아요 개수,수정 날짜
0,[헤어진 날] 꺼내 듣는 노래모음,"['감성', '헤어짐', '위로', '연인', '슬픔', '아픔', '가창력', '...",112336,[헤어진 날] 꺼내 듣는 노래모음,"[413314, 404995, 38662, 492678, 109574, 698250...",12,2017-11-19 13:49:13.000
1,○ 여행&드라이브 차안에서 함께들의면 기분좋아지는 리스트 ○,"['드라이브', '여름', '기분좋은음악', '여행']",61393,○ 여행&드라이브 차안에서 함께들의면 기분좋아지는 리스트 ○,"[426753, 173185, 133128, 155532, 111246, 37479...",487,2016-08-08 12:32:34.000
2,"# 추억의 댄스 가요(운동, 드라이브)","['신나는', '추억', '드라이브', '댄스', '운동', '명곡']",101019,"# 추억의 댄스 가요(운동, 드라이브)","[11777, 61954, 617473, 543751, 274958, 365072,...",621,2018-07-26 00:00:17.000
3,#Sleep,['sleep'],77021,#Sleep,"[626179, 652820, 375317, 164887, 84512, 140833...",13,2020-02-15 13:19:05.000
4,#가을 #비 #밤 #새벽,"['새벽', '가을', '비', '밤']",53094,#가을 #비 #밤 #새벽,"[169984, 118788, 608260, 610311, 536584, 11625...",10,2017-10-30 12:51:02.000


In [ ]:
unique_train.columns

Index(['플레이리스트 제목', '태그 리스트', '플레이리스트 ID', '플레이리스트 제목.1', '곡 리스트', '좋아요 개수',
       '수정 날짜'],
      dtype='object')

In [ ]:
unique_train.rename(columns={'플레이리스트 제목.1':'플레이리스트 태그화'}, inplace=True)

In [ ]:
unique_train.head()

,플레이리스트 제목,태그 리스트,플레이리스트 ID,플레이리스트 태그화,곡 리스트,좋아요 개수,수정 날짜
0,[헤어진 날] 꺼내 듣는 노래모음,"['감성', '헤어짐', '위로', '연인', '슬픔', '아픔', '가창력', '...",112336,[헤어진 날] 꺼내 듣는 노래모음,"[413314, 404995, 38662, 492678, 109574, 698250...",12,2017-11-19 13:49:13.000
1,○ 여행&드라이브 차안에서 함께들의면 기분좋아지는 리스트 ○,"['드라이브', '여름', '기분좋은음악', '여행']",61393,○ 여행&드라이브 차안에서 함께들의면 기분좋아지는 리스트 ○,"[426753, 173185, 133128, 155532, 111246, 37479...",487,2016-08-08 12:32:34.000
2,"# 추억의 댄스 가요(운동, 드라이브)","['신나는', '추억', '드라이브', '댄스', '운동', '명곡']",101019,"# 추억의 댄스 가요(운동, 드라이브)","[11777, 61954, 617473, 543751, 274958, 365072,...",621,2018-07-26 00:00:17.000
3,#Sleep,['sleep'],77021,#Sleep,"[626179, 652820, 375317, 164887, 84512, 140833...",13,2020-02-15 13:19:05.000
4,#가을 #비 #밤 #새벽,"['새벽', '가을', '비', '밤']",53094,#가을 #비 #밤 #새벽,"[169984, 118788, 608260, 610311, 536584, 11625...",10,2017-10-30 12:51:02.000


In [ ]:
a = 'asj194yrsdj@($&#@*!#(!'
for i in string.punctuation:
  a = a.replace(i, '')
print(a)

asj194yrsdj


In [ ]:
unique_train['플레이리스트 태그화'][:5]

0                     [헤어진 날] 꺼내 듣는 노래모음
1      ○ 여행&드라이브 차안에서 함께들의면 기분좋아지는 리스트 ○
2                  # 추억의 댄스 가요(운동, 드라이브)
3                                 #Sleep
4                          #가을 #비 #밤 #새벽
Name: 플레이리스트 태그화, dtype: object

In [ ]:
# 특수문자 제거
clean_columns = []
character_list = ['€', '£', '¥','๑ ❛ ڡ ❛ ๑','♪','♬','ㅇ', 'ෆ', \
                  'ꃼ̫\xa0ꃼ\xa0', '集', '❤', '◟∗❛ᴗ❛∗◞ ', '\xa0\xa0°ټ°\xa0', \
                  '♥', '\xa0͝°\xa0͜ʖ͡°', '❀´▽❀', '°', 'ㅡ', '❣️', '•', 'ㅠ', \
                  '★','☆', "'͡° ͜ʖ ͡°", '○', 'ㅅ' ,'。', '♡', '✓','✔', 'ㅎ', 'ㅋ', '有'\
                  '✨', '＾', '上', '下', "’", '●', '《', 'ㄷ', '♩', '₍₍ ง ˙o˙ว ⁾⁾',\
                  '◕ヮ◕', '☞','☜', '花樣年華', '◇', '♡', '₍ᐢ⑅ᴗ⑅ᐢ₎' 'ㅂ', '┗━━▶','ㄱ',\
                  '！', '＃', '＆', '＊', 'ㅜ', '※','ㅁ','ㄴ','⭐','✋', '「','」', '₍₍ ◝・ω・◟ ⁾⁾',\
                  '˚','✧', '❛˓◞˂̵✧', '' '◀', 'ง ̀́ง', '♨', '~', 'ㄹ', '〰️','ღ', '✌', ' ்́ꇴ ்̀',\
                  '≠', '乃', '―', '♧', '♣', 'ㅉ',
                  '플레이리스트', '리스트', '노래들', '노래', '음악들', '음악', '모음']
import string
columns = unique_train['플레이리스트 태그화']
for i in columns:
    tmp = i.translate(str.maketrans('', '', string.punctuation)).strip()
    for character in character_list:
      tmp = tmp.replace(character,'')
    clean_columns.append(tmp)
print(clean_columns)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
pd.DataFrame(clean_columns)
for i in clean_columns:
  print(i)

In [ ]:
# 한국어 명사화를 위한 라이브러리 
from konlpy.tag import Okt 
# 영어 명사화를 위한 라이브러리 
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
clean_tags = []

for i in clean_columns:
  # 영어는 nltk의 word_tokenize 활용
  if i.isalpha(): 
    tmp = word_tokenize(i)
    clean_tags.append(tmp)
  # 한국어는 konlpy의 Okt 활용
  else:
    tmp = t.nouns(i)
    clean_tags.append(tmp)  

print(clean_tags)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
unique_train['플레이리스트_태그_clean'] = list(clean_tags)

In [ ]:
unique_train.head()

,플레이리스트 제목,태그 리스트,플레이리스트 ID,플레이리스트 태그화,곡 리스트,좋아요 개수,수정 날짜,플레이리스트_태그_clean
0,[헤어진 날] 꺼내 듣는 노래모음,"['감성', '헤어짐', '위로', '연인', '슬픔', '아픔', '가창력', '...",112336,[헤어진 날] 꺼내 듣는 노래모음,"[413314, 404995, 38662, 492678, 109574, 698250...",12,2017-11-19 13:49:13.000,[날]
1,○ 여행&드라이브 차안에서 함께들의면 기분좋아지는 리스트 ○,"['드라이브', '여름', '기분좋은음악', '여행']",61393,○ 여행&드라이브 차안에서 함께들의면 기분좋아지는 리스트 ○,"[426753, 173185, 133128, 155532, 111246, 37479...",487,2016-08-08 12:32:34.000,"[여행, 드라이브, 차안, 의, 기분]"
2,"# 추억의 댄스 가요(운동, 드라이브)","['신나는', '추억', '드라이브', '댄스', '운동', '명곡']",101019,"# 추억의 댄스 가요(운동, 드라이브)","[11777, 61954, 617473, 543751, 274958, 365072,...",621,2018-07-26 00:00:17.000,"[추억, 댄스, 가요, 운동, 드라이브]"
3,#Sleep,['sleep'],77021,#Sleep,"[626179, 652820, 375317, 164887, 84512, 140833...",13,2020-02-15 13:19:05.000,[Sleep]
4,#가을 #비 #밤 #새벽,"['새벽', '가을', '비', '밤']",53094,#가을 #비 #밤 #새벽,"[169984, 118788, 608260, 610311, 536584, 11625...",10,2017-10-30 12:51:02.000,"[가을, 비, 밤, 새벽]"


In [ ]:
unique_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110502 entries, 0 to 110501
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   플레이리스트 제목        110502 non-null  object
 1   태그 리스트           110502 non-null  object
 2   플레이리스트 ID        110502 non-null  int64 
 3   플레이리스트 태그화       110502 non-null  object
 4   곡 리스트            110502 non-null  object
 5   좋아요 개수           110502 non-null  int64 
 6   수정 날짜            110502 non-null  object
 7   플레이리스트_태그_clean  110502 non-null  object
 8   tmp              0 non-null       object
dtypes: int64(2), object(7)
memory usage: 7.6+ MB


In [ ]:
# 태그 합치기
original_tags = unique_train['태그 리스트'][1].lstrip('[').rstrip(']').replace("'", '').split(', ')
additional_tags = unique_train['플레이리스트_태그_clean'][1]

original_tags.extend(additional_tags)
print(original_tags)

['드라이브', '여름', '기분좋은음악', '여행', '여행', '드라이브', '차안', '의', '기분']


In [ ]:
# 태그 합치기
final_list = []
for i in range(0, len(unique_train)):
  original_tags = unique_train['태그 리스트'][i].lstrip('[').rstrip(']').replace("'", '').split(', ')
  additional_tags = unique_train['플레이리스트_태그_clean'][i]
  original_tags.extend(additional_tags)
  final_list.append(set(original_tags))

print(final_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
unique_train['edited_tags'] = final_list

In [ ]:
unique_train.head()

,플레이리스트 제목,태그 리스트,플레이리스트 ID,플레이리스트 태그화,곡 리스트,좋아요 개수,수정 날짜,플레이리스트_태그_clean,tmp,edited_tags
0,[헤어진 날] 꺼내 듣는 노래모음,"['감성', '헤어짐', '위로', '연인', '슬픔', '아픔', '가창력', '...",112336,[헤어진 날] 꺼내 듣는 노래모음,"[413314, 404995, 38662, 492678, 109574, 698250...",12,2017-11-19 13:49:13.000,[날],None,"{감성, 헤어짐, 위로, 연인, 슬픔, 아픔, 가창력, 사랑, 이별, 날}"
1,○ 여행&드라이브 차안에서 함께들의면 기분좋아지는 리스트 ○,"['드라이브', '여름', '기분좋은음악', '여행']",61393,○ 여행&드라이브 차안에서 함께들의면 기분좋아지는 리스트 ○,"[426753, 173185, 133128, 155532, 111246, 37479...",487,2016-08-08 12:32:34.000,"[여행, 드라이브, 차안, 의, 기분]",None,"{기분좋은음악, 의, 여름, 기분, 여행, 드라이브, 차안}"
2,"# 추억의 댄스 가요(운동, 드라이브)","['신나는', '추억', '드라이브', '댄스', '운동', '명곡']",101019,"# 추억의 댄스 가요(운동, 드라이브)","[11777, 61954, 617473, 543751, 274958, 365072,...",621,2018-07-26 00:00:17.000,"[추억, 댄스, 가요, 운동, 드라이브]",None,"{신나는, 추억, 드라이브, 댄스, 가요, 운동, 명곡}"
3,#Sleep,['sleep'],77021,#Sleep,"[626179, 652820, 375317, 164887, 84512, 140833...",13,2020-02-15 13:19:05.000,[Sleep],None,"{sleep, Sleep}"
4,#가을 #비 #밤 #새벽,"['새벽', '가을', '비', '밤']",53094,#가을 #비 #밤 #새벽,"[169984, 118788, 608260, 610311, 536584, 11625...",10,2017-10-30 12:51:02.000,"[가을, 비, 밤, 새벽]",None,"{새벽, 가을, 비, 밤}"


In [ ]:
# 새로운 데이터프레임 생성 후 필요하지 않은 데이터 삭제
tag_df_edited = pd.DataFrame(data = unique_train, columns = ['플레이리스트 ID', '플레이리스트 제목', '태그 리스트', '플레이리스트_태그_clean', 'edited_tags'])
tag_df_edited.columns = ['id', 'plylst_title', 'tags', 'tags_from_playlist','final_tags']
tag_df_edited.head()

,id,plylst_title,tags,tags_from_playlist,final_tags
0,112336,[헤어진 날] 꺼내 듣는 노래모음,"['감성', '헤어짐', '위로', '연인', '슬픔', '아픔', '가창력', '...",[날],"{감성, 헤어짐, 위로, 연인, 슬픔, 아픔, 가창력, 사랑, 이별, 날}"
1,61393,○ 여행&드라이브 차안에서 함께들의면 기분좋아지는 리스트 ○,"['드라이브', '여름', '기분좋은음악', '여행']","[여행, 드라이브, 차안, 의, 기분]","{기분좋은음악, 의, 여름, 기분, 여행, 드라이브, 차안}"
2,101019,"# 추억의 댄스 가요(운동, 드라이브)","['신나는', '추억', '드라이브', '댄스', '운동', '명곡']","[추억, 댄스, 가요, 운동, 드라이브]","{신나는, 추억, 드라이브, 댄스, 가요, 운동, 명곡}"
3,77021,#Sleep,['sleep'],[Sleep],"{sleep, Sleep}"
4,53094,#가을 #비 #밤 #새벽,"['새벽', '가을', '비', '밤']","[가을, 비, 밤, 새벽]","{새벽, 가을, 비, 밤}"


In [ ]:
tag_df_edited.to_csv('tag_list_edited.csv',encoding='utf-8')

In [9]:
tag_df_edited = pd.read_csv('/content/drive/MyDrive/data/카카오아레나_멜론/tag_list_edited.csv',index_col=0)

## 1주차 - 데이터 전처리 (윤식씨)

In [11]:
train_3 = pd.read_csv('/content/drive/MyDrive/data/카카오아레나_멜론/train_3.csv',index_col=0)

In [12]:
train_3.head()

,plylst_title,tags,id,plylst_title.1,songs,like_cnt,updt_date,genre,genre_detail,genre_name,genre_detail_name
0,[헤어진 날] 꺼내 듣는 노래모음,"['연인', '감성', '이별', '위로', '슬픔', '가창력', '헤어짐', '...",112336,[헤어진 날] 꺼내 듣는 노래모음,"[413314, 404995, 38662, 492678, 109574, 698250...",12,2017-11-19 13:49:13.000,"['GN0400', 'GN1500', 'GN2500', 'GN0300', 'GN01...","['GN0301', 'GN1504', 'GN0104', 'GN2501', 'GN04...",['R&B/Soul' 'OST' '아이돌' '랩/힙합' '발라드'],"[""'00"" ""'10-"" 'R&B' '국내드라마' '남자 아이돌' '발라드' '보컬..."
1,○ 여행&드라이브 차안에서 함께들의면 기분좋아지는 리스트 ○,"['여행', '드라이브', '여름', '기분좋은음악', 'R&B/Soul', 'OS...",61393,○ 여행&드라이브 차안에서 함께들의면 기분좋아지는 리스트 ○,"[426753, 173185, 133128, 155532, 111246, 37479...",487,2016-08-08 12:32:34.000,"['GN0400', 'GN1500', 'GN0500', 'GN0200', 'GN25...","['GN0606', 'GN0105', 'GN0303', 'GN2505', 'GN15...",['R&B/Soul' 'OST' '인디음악' '댄스' '아이돌' '발라드' '랩/힙...,"[""'10-"" 'R&B' '국내드라마' '록' '발라드' '보컬 스타일' '어반' ..."
2,"# 추억의 댄스 가요(운동, 드라이브)","['운동', '댄스', '추억', '드라이브', '신나는', '명곡', '댄스', ...",101019,"# 추억의 댄스 가요(운동, 드라이브)","[11777, 61954, 617473, 543751, 274958, 365072,...",621,2018-07-26 00:00:17.000,"['GN0200', 'GN2500', 'GN0300', 'GN0600', 'GN01...","['GN0702', 'GN0301', 'GN0604', 'GN2503', 'GN01...",['댄스' '아이돌' '랩/힙합' '록/메탈' '발라드' '성인가요'],"[""'00"" ""'90"" '남자 아이돌' '댄스' '랩 스타일' '랩/힙합' '보컬 ..."
3,#Sleep,"['sleep', '포크/블루스', 'R&B/Soul', 'OST', '인디음악',...",77021,#Sleep,"[626179, 652820, 375317, 164887, 84512, 140833...",13,2020-02-15 13:19:05.000,"['GN0800', 'GN0400', 'GN1500', 'GN0500', 'GN02...","['GN0606', 'GN0905', 'GN1302', 'GN0105', 'GN09...",['포크/블루스' 'R&B/Soul' 'OST' '인디음악' '댄스' 'J-POP'...,"[""'00"" ""'00-'10"" ""'10-"" ""'60-'70"" 'R&B' '국내드라마..."
4,#가을 #비 #밤 #새벽,"['비', '가을', '밤', '새벽', '포크/블루스', 'R&B/Soul', '...",53094,#가을 #비 #밤 #새벽,"[169984, 118788, 608260, 610311, 536584, 11625...",10,2017-10-30 12:51:02.000,"['GN0800', 'GN0400', 'GN1500', 'GN0500', 'GN02...","['GN0606', 'GN1302', 'GN0105', 'GN0901', 'GN13...",['포크/블루스' 'R&B/Soul' 'OST' '인디음악' '댄스' '록/메탈' ...,"[""'00"" ""'10-"" 'R&B' '국내드라마' '국외영화' '남자 아이돌' '댄..."


In [10]:
tag_df_edited

,id,plylst_title,tags,tags_from_playlist,final_tags
0,112336,[헤어진 날] 꺼내 듣는 노래모음,"['감성', '헤어짐', '위로', '연인', '슬픔', '아픔', '가창력', '...",['날'],"{'감성', '헤어짐', '위로', '연인', '슬픔', '아픔', '가창력', '..."
1,61393,○ 여행&드라이브 차안에서 함께들의면 기분좋아지는 리스트 ○,"['드라이브', '여름', '기분좋은음악', '여행']","['여행', '드라이브', '차안', '의', '기분']","{'기분좋은음악', '의', '여름', '기분', '여행', '드라이브', '차안'}"
2,101019,"# 추억의 댄스 가요(운동, 드라이브)","['신나는', '추억', '드라이브', '댄스', '운동', '명곡']","['추억', '댄스', '가요', '운동', '드라이브']","{'신나는', '추억', '드라이브', '댄스', '가요', '운동', '명곡'}"
3,77021,#Sleep,['sleep'],['Sleep'],"{'sleep', 'Sleep'}"
4,53094,#가을 #비 #밤 #새벽,"['새벽', '가을', '비', '밤']","['가을', '비', '밤', '새벽']","{'새벽', '가을', '비', '밤'}"
...,...,...,...,...,...
110497,150423,＊팝으로말하는 사랑 이야기＊,['팝'],"['팝', '말', '사랑', '이야기']","{'사랑', '말', '팝', '이야기'}"
110498,38150,＊＊＊,"['힙합', '랩']",[],"{'힙합', '랩'}"
110499,89052,＊＊＊nb＊＊＊,"['힙합', '랩']",['nb'],"{'nb', '힙합', '랩'}"
110500,13832,＿Now and Here,['락'],[],{'락'}


In [32]:
merge_df = pd.merge(tag_df_edited,train_3[['id','genre_name','genre_detail_name']], on='id')

In [36]:
merge_df.head()

,id,plylst_title,tags,tags_from_playlist,final_tags,genre_name,genre_detail_name
0,112336,[헤어진 날] 꺼내 듣는 노래모음,"['감성', '헤어짐', '위로', '연인', '슬픔', '아픔', '가창력', '...",['날'],"[감성, 헤어짐, 위로, 연인, 슬픔, 아픔, 가창력, 사랑, 이별, 날]","[R&B/Soul, OST, 아이돌, 랩/힙합, 발라드]","[00, 10-, R&B, 국내드라마, 남자, 아이돌, 발라드, 보컬, 스타일]"
1,61393,○ 여행&드라이브 차안에서 함께들의면 기분좋아지는 리스트 ○,"['드라이브', '여름', '기분좋은음악', '여행']","['여행', '드라이브', '차안', '의', '기분']","[기분좋은음악, 의, 여름, 기분, 여행, 드라이브, 차안]","[R&B/Soul, OST, 인디음악, 댄스, 아이돌, 발라드, 랩/힙합, 록/메탈...","[10-, R&B, 국내드라마, 록, 발라드, 보컬, 스타일, 어반, 언더그라운드,..."
2,101019,"# 추억의 댄스 가요(운동, 드라이브)","['신나는', '추억', '드라이브', '댄스', '운동', '명곡']","['추억', '댄스', '가요', '운동', '드라이브']","[신나는, 추억, 드라이브, 댄스, 가요, 운동, 명곡]","[댄스, 아이돌, 랩/힙합, 록/메탈, 발라드, 성인가요]","[00, 90, 남자, 아이돌, 댄스, 랩, 스타일, 랩/힙합, 보컬, 스타일, 신..."
3,77021,#Sleep,['sleep'],['Sleep'],"[sleep, Sleep]","[포크/블루스, R&B/Soul, OST, 인디음악, 댄스, J-POP, 아이돌, ...","[00, 00-10, 10-, 60-70, R&B, 국내드라마, 댄스, 록, 발라드..."
4,53094,#가을 #비 #밤 #새벽,"['새벽', '가을', '비', '밤']","['가을', '비', '밤', '새벽']","[새벽, 가을, 비, 밤]","[포크/블루스, R&B/Soul, OST, 인디음악, 댄스, 록/메탈, 아이돌, 랩...","[00, 10-, R&B, 국내드라마, 국외영화, 남자, 아이돌, 댄스, 랩, 스타..."


In [33]:
merge_df['final_tags'] = merge_df['final_tags'].apply(lambda x : x[1:-1].replace("'",'').replace(' ','').split(','))

In [34]:
merge_df['genre_name'] = merge_df['genre_name'].apply(lambda x : x[1:-1].replace("'",'').split(' '))

In [35]:
merge_df['genre_detail_name'] = merge_df['genre_detail_name'].apply(lambda x : x[1:-1].replace("'",'').replace('"','').split(' '))

In [37]:
merge_df.apply(lambda x : x.final_tags.extend(x.genre_name),axis=1)

0         None
1         None
2         None
3         None
4         None
          ... 
110497    None
110498    None
110499    None
110500    None
110501    None
Length: 110502, dtype: object

In [38]:
merge_df.apply(lambda x : x.final_tags.extend(x.genre_detail_name),axis=1)

0         None
1         None
2         None
3         None
4         None
          ... 
110497    None
110498    None
110499    None
110500    None
110501    None
Length: 110502, dtype: object

In [39]:
merge_df.final_tags[3]

['sleep',
 'Sleep',
 '포크/블루스',
 'R&B/Soul',
 'OST',
 '인디음악',
 '댄스',
 'J-POP',
 '아이돌',
 '랩/힙합',
 '록/메탈',
 '뉴에이지\n',
 'R&B/Soul',
 '발라드',
 'POP',
 '00',
 '00-10',
 '10-',
 '60-70',
 'R&B',
 '국내드라마',
 '댄스',
 '록',
 '발라드',
 '보컬',
 '스타일\n',
 '시부야케이',
 '어반',
 '여자',
 '아이돌',
 '이지리스닝',
 '일렉트로니카',
 '컨템포러리',
 'R&B',
 '컴필레이션',
 '포크',
 '피아노']

In [40]:
merge_df.to_json('/content/drive/MyDrive/data/카카오아레나_멜론/final_tag.json')